# Image Classification Using Feature Extraction and YOLO Model

In [ ]:
# Import analysis functions and YOLO
import pathlib
import numpy as np
from analysis import (
    standardize_image, get_avg_color, get_contrast_level, detect_edges
)
from ultralytics import YOLO
import os

In [ ]:
# Load YOLO model for comparison with feature-based classification
model_path = pathlib.Path(__file__).parent.parent / 'machinelearning' / 'yolo11clsFineTuned.pt'
try:
    yolo_model = YOLO(str(model_path))
    print("YOLO model loaded successfully")
except Exception as e:
    print(f"Could not load YOLO model: {e}")
    yolo_model = None

In [ ]:
# Load image paths from dataset directory
dataset_dir = pathlib.Path(__file__).parent.parent / 'dataSet' / 'Data'
image_paths = [p for p in dataset_dir.glob('**/*') if p.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp']]
print(f"Found {len(image_paths)} images.")

In [ ]:
# Extract features for each image and get YOLO predictions
features = []
for path in image_paths:
    img = standardize_image(path)
    avg_color = get_avg_color(img)
    contrast = get_contrast_level(img)
    edge_info = detect_edges(img, method='canny')
    
    # YOLO prediction
    yolo_prediction = 'unknown'
    yolo_confidence = 0.0
    if yolo_model:
        try:
            results = yolo_model.predict(str(path), verbose=False)
            if results and len(results) > 0 and hasattr(results[0], 'probs'):
                class_id = results[0].probs.top1
                confidence = float(results[0].probs.top1conf)
                class_names = results[0].names if hasattr(results[0], 'names') else {0: 'clean', 1: 'dirty'}
                yolo_prediction = class_names.get(class_id, 'unknown')
                yolo_confidence = confidence
        except Exception as e:
            print(f"YOLO prediction failed for {path}: {e}")
    
    features.append({
        'path': str(path),
        'avg_color': avg_color,
        'contrast': contrast,
        'edge_density': edge_info['statistics']['edge_density'],
        'yolo_prediction': yolo_prediction,
        'yolo_confidence': yolo_confidence
    })
print(f"Extracted features for {len(features)} images.")

## Define Classification Thresholds
We will use simple thresholds on average brightness, contrast, and edge density to classify images into two classes:

In [ ]:
# Example thresholds (tune as needed for your dataset)
BRIGHTNESS_THRESHOLD = 100
CONTRAST_THRESHOLD = 50
EDGE_DENSITY_THRESHOLD = 0.05

In [ ]:
# Classify images based on thresholds
results = []
for feat in features:
    brightness = feat['avg_color'].get('brightness', feat['avg_color'].get('average_gray', 0))
    contrast = feat['contrast'].get('global_contrast', feat['contrast']).get('contrast_level', 0)
    edge_density = feat['edge_density']
    
    # Example rule-based classification
    if brightness > BRIGHTNESS_THRESHOLD and contrast > CONTRAST_THRESHOLD and edge_density > EDGE_DENSITY_THRESHOLD:
        label = 'Class A'
    else:
        label = 'Class B'
    
    results.append({
        'path': feat['path'],
        'brightness': brightness,
        'contrast': contrast,
        'edge_density': edge_density,
        'label': label
    })
print(f"Classified {len(results)} images.")

In [ ]:
# Display classification results
import pandas as pd
df = pd.DataFrame(results)
display(df.head())

In [ ]:
# Compare YOLO predictions with rule-based classification
comparison_results = []
for feat, result in zip(features, results):
    comparison_results.append({
        'path': feat['path'],
        'rule_based_label': result['label'],
        'yolo_prediction': feat['yolo_prediction'],
        'yolo_confidence': feat['yolo_confidence'],
        'brightness': result['brightness'],
        'contrast': result['contrast'],
        'edge_density': result['edge_density']
    })

# Display comparison
import pandas as pd
comparison_df = pd.DataFrame(comparison_results)
display(comparison_df.head(10))